# init

In [1]:
from quantifiers.CC import classify_count
from quantifiers.ACC import ACC
from quantifiers.dys_method import dys_method
from quantifiers.MS import MS_method

from utils.getTrainingScores import getTrainingScores
from utils.getTPRFPR import getTPRFPR
from utils.applyquantifiers import apply_quantifier
from utils.fitQuantifierSchumacherGithub import fitQuantifierSchumacherGithub

import pdb
import quapy as qp
import os
import numpy as np
import pandas as pd
import joblib
from scipy.io.arff import loadarff
from pprint import pprint
import matplotlib.pyplot as plt
import time
import math

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.calibration import CalibrationDisplay

meta_table_path = './metafeatures/meta-features-table.csv'
path = "./datasets/"
# files = os.listdir(path)
meta_features_table_index = pd.read_csv('./metafeatures/meta-features-table-index.csv')
files = meta_features_table_index.pop('dataset_name').tolist()

counters = ['CC', 'ACC', 'PACC', 'PCC', 'SMM', 'HDy', 'DyS', 'SORD', 'MS', 'MS2', 'MAX', 'X', 'T50']
# counters = ["CC","ACC","SMM","HDy","DyS","SORD","MS","MS2","MAX","X","T50","PCC","PACC","GAC","GPAC","FM"]

# preprocess

In [2]:
i = 0
dataframe = None
X = None
y = None
X_list = []
y_list = []

for f in files:
  df = pd.read_csv(path + f)
  df = df.dropna()
  
  y = df.pop(df.columns[-1])
  X = df

  y_list.append(y.to_numpy())
  X_list.append(X.to_numpy())

  i += 1
i = 0

In [3]:
meta_features_table = pd.read_csv(meta_table_path)

if os.path.isfile('./metafeatures/meta-table.csv'):
    meta_table = pd.read_csv('./metafeatures/meta-table.csv')
else:
    meta_table_columns = meta_features_table.columns.tolist()
    for counter in counters:
        meta_table_columns.append('arr_' + counter)
    meta_table = pd.DataFrame(columns=meta_table_columns)

In [4]:
meta_features_table

,attr_conc.mean,attr_conc.sd,attr_ent.mean,attr_ent.sd,attr_to_inst,best_node.mean,best_node.sd,can_cor.mean,can_cor.sd,cat_to_num,...,tree_imbalance.sd,tree_shape.mean,tree_shape.sd,var.mean,var.sd,var_importance.mean,var_importance.sd,w_lambda,worst_node.mean,worst_node.sd
0,0.008139,0.048213,0.004126,0.490599,0.004135,0.561303,0.002908,0.561218,0.0,0.0,...,0.337663,0.012390,0.179874,7.776980e-34,7.133986e-34,0.038708,0.069519,0.647064,0.556616,0.004692
1,0.106477,0.213625,0.430605,0.412370,0.008805,0.204547,0.454586,0.279843,0.0,0.0,...,0.580063,0.100662,0.342257,1.675777e-32,1.081000e-32,0.246988,0.132723,0.890612,0.189155,0.340122
2,0.116658,0.375055,0.526600,0.470566,0.010323,0.784169,0.010684,0.528722,0.0,0.0,...,0.484229,0.099970,0.413085,4.701315e-29,8.591731e-29,0.050255,0.085777,0.682031,0.785998,0.017235
3,0.137671,0.461136,0.555110,0.391798,0.009623,0.819016,0.001322,0.394100,0.0,0.0,...,0.443044,0.071106,0.380437,1.538522e-27,2.812553e-27,0.050255,0.041365,0.807865,0.821864,0.002133
4,0.276312,0.377796,0.362371,0.208240,0.099335,0.431910,0.464158,0.602796,0.0,0.0,...,0.631564,0.212819,0.834952,2.406547e-28,4.516420e-28,0.047472,0.055058,0.599718,0.409391,0.271584
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,0.074462,0.392015,0.199253,0.657376,0.001311,0.729914,0.244046,0.826111,0.0,0.0,...,0.341897,0.010386,0.094861,5.138459e-38,2.347324e-38,0.013001,0.070563,0.295391,0.486346,0.140438
129,0.098333,0.152756,0.879157,0.633259,0.000006,0.980050,0.035743,0.927209,0.0,0.0,...,0.411869,0.053137,0.305895,1.248782e-32,5.719199e-33,0.397590,0.838834,0.129877,0.561986,0.302595
130,0.064769,0.073679,0.830121,0.078607,0.000286,0.564553,0.075999,0.452016,0.0,0.0,...,0.303553,0.022339,0.166031,4.423299e-37,1.811357e-45,0.397590,0.168393,0.757488,0.534034,0.064233
131,0.057703,0.304893,0.125438,0.565499,0.004991,0.596928,0.234005,0.725345,0.0,0.0,...,0.351968,0.031128,0.271067,3.156065e-33,6.517244e-33,0.031213,0.082105,0.443156,0.301793,0.093962


In [5]:
# # meta_table

# meta_table.loc[len(meta_table.index)] = meta_features_table.iloc[0].tolist()
# meta_table.loc[len(meta_table.index)] = meta_features_table.iloc[1].tolist()
# meta_table.loc[len(meta_table.index)] = meta_features_table.iloc[2].tolist()
# meta_table.loc[len(meta_table.index)] = meta_features_table.iloc[3].tolist()
# meta_table.loc[len(meta_table.index)] = meta_features_table.iloc[4].tolist()

# meta_table

In [6]:
# meta_table_columns = meta_features_table.columns.tolist()
# for counter in counters:
#     meta_table_columns.append('arr_' + counter)
# meta_table_columns

# meta-features

In [7]:
def run_experiment(X, y, dataset_name):
  #......................input/output path directories....................

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
  
  rf_clf = None
  rf_clf = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)
  
  calib_clf = CalibratedClassifierCV(rf_clf, cv=3, n_jobs=-1)
  calib_clf.fit(X_train, y_train)

  # try:
  #   rf_clf = joblib.load('./models/' + dataset_name + '.joblib')
  #   rf_clf.n_jobs = -1
  # except:
  #   rf_clf = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)

  scores = getTrainingScores(X_train, y_train, 10, rf_clf)[0] # None
  tprfpr = getTPRFPR(scores)
  rf_clf.fit(X_train, y_train)

  # disp = CalibrationDisplay.from_estimator(rf_clf, X_test, y_test)
  # disp.plot()

  # disp = CalibrationDisplay.from_estimator(calib_clf, X_test, y_test)
  # disp.plot()

  # plt.show()

  niterations = 10 # how many replicates it will take
  batch_sizes = list([10]) # list(range(10, min(91, max_allowed + 1), 10))# + list(range(100, min(501, max_allowed + 1), 100)) # test set sizes
  alpha_values = [round(x, 2) for x in np.linspace(0,1,20)]   # class proportion
  alpha_values = [x for x in alpha_values if x < 0.4 or x > 0.6]  # removing alpha > 0.4 and alpha < 0.6

  pos_scores = scores[scores["class"]==1]["scores"]
  neg_scores = scores[scores["class"]==0]["scores"]
  
  X_test = pd.DataFrame(X_test)
  y_test = pd.DataFrame(y_test, columns=[str(len(X_test.columns))])
  df_test = pd.concat([X_test, y_test], axis=1)
  
  # WAS ZERO (0) BEFORE
  df_test_pos = df_test.loc[df_test[df_test.columns[-1]] == 1] # seperating positive test examples
  df_test_neg = df_test.loc[df_test[df_test.columns[-1]] == 0] # seperating negative test examples
  
  # pdb.set_trace()

  table=pd.DataFrame(columns=['quantifier', 'abs-error', 'execution-time'])
  for sample_size in batch_sizes:   # [10,100,500], batch_sizes, Varying test set sizes

    for alpha in alpha_values: # Varying positive class distribution
      # abs_error_cc = []
      # abs_error_dys = []

      error = []

      for iter in range(niterations):
        pos_size = int(round(sample_size * alpha, 2))
        neg_size = sample_size - pos_size

        #
        # AVISAR O PROF ANDRE QUE SÓ FUNCIONOU COM REPLACE = TRUE
        #
        # df_test_neg
        # sample_test_pos = df_test_pos.sample( int(pos_size), replace = False)
        # sample_test_neg = df_test_neg.sample( int(neg_size), replace = False)
        sample_test_pos = df_test_pos.sample( int(pos_size), replace = True)
        sample_test_neg = df_test_neg.sample( int(neg_size), replace = True)

        sample_test = pd.concat([sample_test_pos, sample_test_neg])

        test_label = sample_test[sample_test.columns[-1]] # sample_test["class"]

        test_sample = sample_test.drop([sample_test.columns[-1]], axis=1) # sample_test.drop(["class"], axis=1)  #dropping class label columns
        te_scores = rf_clf.predict_proba(test_sample)[:,1]  #estimating test sample scores

        n_pos_sample_test = list(test_label).count(1) #Counting num of actual positives in test sample
        calcultd_pos_prop = round(n_pos_sample_test/len(sample_test), 2) #actual pos class prevalence in generated sample

        # print(counters)
        for quantifier in counters:
          #..............Test Sample QUAPY exp...........................
          te_quapy = None
          external_qnt = None
          #if quantifier in ['EM', 'PWK']:
          #  print('ok')
          #  external_qnt = fitQuantifierSchumacherGithub(quantifier, X_train, y_train)                  
          #  te_quapy = qp.data.LabelledCollection(sample_test.drop(["class","Binary_label"], axis=1), test_label)
          
          #.............Calling of Methods..................................................
          start = time.time()
          pred_pos_prop = apply_quantifier(qntMethod=quantifier,
                                           clf=calib_clf,
                                           scores=scores,
                                           p_score=pos_scores,
                                           n_score=neg_scores,
                                           train_labels=None,
                                           test_score=te_scores,
                                           TprFpr=tprfpr,
                                           thr=0.5,
                                           measure='hellinger',
                                           test_data=test_sample,
                                           test_quapy=te_quapy,
                                           external_qnt=external_qnt) #y_test=test_label
          stop = time.time()
          t = stop - start

          pred_pos_prop = np.round(pred_pos_prop,2)  #predicted class proportion
          
          #..............................RESULTS Evaluation.....................................
          abs_error = round(abs(calcultd_pos_prop - pred_pos_prop), 2) # absolute error
          
          table.loc[len(table)] = [quantifier,abs_error,t]
          # error = round(calcultd_pos_prop - pred_pos_prop, 2)     # simple error Biasness
          
  return table

In [8]:
clf = RandomForestClassifier(random_state=42)
skip_count = 0
table = None
# warnings.filterwarnings('ignore')

result = {}
for counter in counters:
    result[counter] = []

file = open('log.txt', 'w')
file.close()

result_index = 0
for i in range(len(meta_table), len(X_list)):
  try:
    clf = RandomForestClassifier()

    table = run_experiment(X_list[i], y_list[i], str(files[i].split('.csv')[0]))

    table = table.groupby('quantifier')['abs-error', 'execution-time'].aggregate('mean')

    alpha = 0
    for key in counters:
      try:
        sum = 0
        for k in counters:
          if k != key:
            sum += ((1 - table['abs-error'][key]) / (1 - table['abs-error'][k]))
            # sum += ((1 - table['abs-error'][key]) / (1 - table['abs-error'][k])) / (1 + alpha * math.log((table['execution-time'][key] / table['execution-time'][k])))
        arr = sum / (len(counters) - 1)
        result[key].append(arr)
      except:
        result[key].append(-1)    

    row = meta_features_table.iloc[i].tolist()
    for key in result:
      row.append(result[key][result_index])
    meta_table.loc[len(meta_table.index)] = row
    
    meta_table.to_csv('./metafeatures/meta-table.csv', index = False)

    result_index += 1
    print('Finished ' + str(i))
  except Exception as e:
    print('Skipping ' + str(i) + '...\t\t\t' + str(e))
    # for key in meta_table_dict:
    #   meta_table_dict[key].drop(i, inplace = True)
    skip_count += 1
    
    file = open('log.txt', 'a')
    file.write('Skipping ' + str(i) + '...\t\t\t' + str(e) + '\n')
    file.write('Dataset: ' + str(files[i]) + '\n')
    file.write('table[abs-error][key]:\n' + str(table['abs-error'][key]) + '\n')
    file.write('table[abs-error][k]:\n' + str(table['abs-error'][k]) + '\n')
    file.write('table[execution-time][key]:\n' + str(table['execution-time'][key]) + '\n')
    file.write('table[execution-time][k]:\n' + str(table['execution-time'][k]) + '\n')
    file.write(str(table['execution-time'][key] / table['execution-time'][k]) + '\n')
    file.write('\n')
    file.close()


  # # # TEST
  # # if i == 6:
  # #   break

# for key in meta_table_dict:
#   meta_table_dict[key]['arr'] = result[key]

# for key in result:
#     meta_features_table[('arr_' + key)] = result[key]

print('\n\nSkipped ' + str(skip_count) + ' dataset(s)')



Skipped 0 dataset(s)


In [9]:
# result

In [10]:
# meta_features_table.to_csv('./metafeatures/meta-table.csv', index = False)

In [11]:
# result

In [12]:
# import pickle

# with open('result.pkl', 'wb') as fp:
#     pickle.dump(result, fp)
#     print('dictionary saved successfully to file')

In [13]:
# with open('result.pkl', 'rb') as fp:
#     person = pickle.load(fp)
#     print(person)

In [14]:
# result
# meta_table_dict

In [15]:
# finSavePath = './metafeatures/meta-table'
# for c in counters:
#     meta_table_dict[c].to_csv(str(finSavePath + '-' + c + '.csv'), index = False)

In [16]:
# import pandas as pd

# test = pd.read_csv('./metafeatures/meta-table-PACC.csv')
# test = test.pop(test.columns[-1])
# test

In [17]:
# import numpy as np

# i = 0
# for t in test:
#     if np.isnan(t):
#         print(i)
#     i += 1